### Optimal Tactical Pathing for UGV

In [24]:
import gurobipy as gp
from gurobipy import Model, GRB
import numpy as np
import scipy.sparse as sp
import pandas as pd
import os
import csv
from PIL import Image

In [26]:
SITUATION = "Buckner"
SPACING = 40

In [27]:
sat_path = os.path.join(os.getcwd(), 'imagery', SITUATION+'.png')
sat_map = np.asarray(Image.open(sat_path))
l, w, h = sat_map.shape
rows = l // SPACING
cols = w // SPACING
cols


3

In [28]:
arcs = pd.read_csv('Buckner_arcs_3_3.csv',header=None)
arcs
ins_csv = pd.read_csv('Buckner_ins_3_3.csv', header=None)
outs_csv = pd.read_csv('Buckner_outs_3_3.csv', header=None)

arcs


,0,1,2,3,4,5,6
0,1,1,2,0.005661,179.104478,-0.443756,0
1,2,1,5,1.000000,179.104478,232.573500,0
2,3,1,4,0.005316,179.104478,352.700609,0
3,4,1,10,0.050055,179.104478,0.000000,1
4,5,2,3,0.005152,179.104478,260.180215,0
...,...,...,...,...,...,...,...
93,94,17,8,0.005866,179.104478,0.000000,0
94,95,18,17,0.047182,179.104478,-111.114877,1
95,96,18,14,1.000000,179.104478,-57.404831,1
96,97,18,15,0.051187,179.104478,-115.663070,1


In [29]:
inflow = ins_csv.apply(lambda row: row[row != 0].tolist(), axis=1).to_dict()
inflow = {
    idx + 1: row[row != 0].tolist()
    for idx, row in ins_csv.iterrows()
}


outflow = outs_csv.apply(lambda row: row[row != 0].tolist(), axis=1).to_dict()
outflow = {
    idx + 1: row[row != 0].tolist()
    for idx, row in outs_csv.iterrows()
}

In [30]:
inflow

{1: [9, 18, 26, 53],
 2: [1, 13, 19, 27, 33, 59],
 3: [5, 28, 34, 63],
 4: [3, 8, 25, 37, 42, 69],
 5: [2, 7, 12, 15, 32, 38, 43, 47, 78],
 6: [6, 11, 21, 44, 48, 84],
 7: [17, 24, 41, 88],
 8: [16, 23, 31, 36, 46, 94],
 9: [22, 30, 40, 98],
 10: [4, 58, 67, 75],
 11: [10, 50, 62, 68, 76, 82],
 12: [14, 54, 77, 83],
 13: [20, 52, 57, 74, 86, 91],
 14: [29, 51, 56, 61, 64, 81, 87, 92, 96],
 15: [35, 55, 60, 70, 93, 97],
 16: [39, 66, 73, 90],
 17: [45, 65, 72, 80, 85, 95],
 18: [49, 71, 79, 89]}

In [31]:
t = 60*60*60
s = 1
f = 9
N=cols*rows*2
A=max(arcs[0])
d = arcs[3]
E = arcs[5]
t = 60*60*60

batteryCapacity = 100000
A

98

In [77]:
print("Inflow:", inflow)
print("Outflow:", outflow)

Inflow: {1: [9, 18, 26, 53], 2: [1, 13, 19, 27, 33, 59], 3: [5, 28, 34, 63], 4: [3, 8, 25, 37, 42, 69], 5: [2, 7, 12, 15, 32, 38, 43, 47, 78], 6: [6, 11, 21, 44, 48, 84], 7: [17, 24, 41, 88], 8: [16, 23, 31, 36, 46, 94], 9: [22, 30, 40, 98], 10: [4, 58, 67, 75], 11: [10, 50, 62, 68, 76, 82], 12: [14, 54, 77, 83], 13: [20, 52, 57, 74, 86, 91], 14: [29, 51, 56, 61, 64, 81, 87, 92, 96], 15: [35, 55, 60, 70, 93, 97], 16: [39, 66, 73, 90], 17: [45, 65, 72, 80, 85, 95], 18: [49, 71, 79, 89]}
Outflow: {1: [1, 2, 3, 4], 2: [5, 6, 7, 8, 9, 10], 3: [11, 12, 13, 14], 4: [15, 16, 17, 18, 19, 20], 5: [21, 22, 23, 24, 25, 26, 27, 28, 29], 6: [30, 31, 32, 33, 34, 35], 7: [36, 37, 38, 39], 8: [40, 41, 42, 43, 44, 45], 9: [46, 47, 48, 49], 10: [50, 51, 52, 53], 11: [54, 55, 56, 57, 58, 59], 12: [60, 61, 62, 63], 13: [64, 65, 66, 67, 68, 69], 14: [70, 71, 72, 73, 74, 75, 76, 77, 78], 15: [79, 80, 81, 82, 83, 84], 16: [85, 86, 87, 88], 17: [89, 90, 91, 92, 93, 94], 18: [95, 96, 97, 98]}


In [49]:
from gurobipy import Model, GRB

def two_step_optimization(w, h, s, f, A, N, d, t, E, inflow, outflow, batteryCapacity):
    MAXTIME = 120  # Time limit for optimization
    MIPGAP = 1e-2  # Allowable MIP gap

    if isinstance(A, int):  
        A = list(range(1, A+ 1))

    print(A)

    # Step 1: Minimize detection cost
    m1 = Model("Least_Detection_Path")
    m1.setParam("TimeLimit", MAXTIME)
    m1.setParam("MIPGap", MIPGAP)
    m1.setParam("OutputFlag", 1)

    # Decision variables: x[i] = 1 if arc i is used, 0 otherwise
    x = m1.addVars(A, vtype=GRB.BINARY, name="x")

    # Objective: Minimize detection cost
    m1.setObjective(sum(d[i] * x[i] for i in A), GRB.MINIMIZE)

    # Debug: Print node count
    print(f"Total Nodes (N): {N}")

    # Ensure start and finish conditions
    if s in inflow and s in outflow:
        m1.addConstr(sum(x[k] for k in inflow.get(s, [])) - sum(x[k] for k in outflow.get(s, [])) == -1)
    
    if f in inflow and f in outflow:
        m1.addConstr(sum(x[k] for k in inflow.get(f, [])) - sum(x[k] for k in outflow.get(f, [])) == 1)

    # Flow conservation constraints (for intermediate nodes)
    for i in range(1, N + 1):
        inflow_arcs = inflow.get(i, [])
        outflow_arcs = outflow.get(i, [])
        
        m1.addConstr(sum(x[k] for k in inflow_arcs) <= 1)
        m1.addConstr(sum(x[k] for k in outflow_arcs) <= 1)
        
        if i != s and i != f:
            m1.addConstr(sum(x[k] for k in inflow_arcs) - sum(x[k] for k in outflow_arcs) == 0)

    # Time constraint
    m1.addConstr(sum(t[i] * x[i] for i in A) <= E)  # Assuming E is the time limit (τ)

    # Solve model
    m1.optimize()

    # Extract solution
    if m1.status == GRB.OPTIMAL:
        optimal_path = [i for i in A if x[i].x > 0.5]
        path_cost = m1.objVal
    else:
        optimal_path = []
        path_cost = None

    print("Least detection path:", optimal_path)
    print("Detection cost:", path_cost)

    return optimal_path, path_cost

def write_path(opt_path, scenario_name="default"):
    path_df = pd.DataFrame(opt_path, columns=["Path"])
    path_df.to_csv(f"output_{scenario_name}.csv", index=False)
    print(f"Path saved to output_{scenario_name}.csv")


In [45]:
print(f"Type of inflow: {type(inflow)}")  
print(f"Type of outflow: {type(outflow)}")  

print(f"Inflow sample: {inflow.get(s, 'Not found')}")  
print(f"Outflow sample: {outflow.get(f, 'Not found')}")  



Type of inflow: <class 'dict'>
Type of outflow: <class 'dict'>
Inflow sample: [9, 18, 26, 53]
Outflow sample: [46, 47, 48, 49]


In [42]:
print(f"Nodes in inflow: {list(inflow.keys())}")
print(f"Nodes in outflow: {list(outflow.keys())}")


Nodes in inflow: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Nodes in outflow: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [44]:
# Assume arcs is a DataFrame with columns like ['arc_id', 'start_node', 'end_node']
f = arcs[98].max()  # Setting f to the largest node number

KeyError: 98

In [50]:
two_step_optimization(w, h, s, f, A, N, d, t, E, inflow, outflow, batteryCapacity)

Set parameter TimeLimit to value 120
Set parameter MIPGap to value 0.01
Set parameter OutputFlag to value 1


KeyError: 98